# PP S3 Query Demo
* StellarAlgo Data Science
* Peter Morrison
* June 27, 2022

This notebook demonstrates 3 ways to query S3

To read these from Legacy environment you will need to asssume the Explore-US role. See an example of assuming the Legacy role here https://github.com/stellaralgo/data-sci-retention/blob/main/lambdas/inference/post_process_scores/post_process_scores.py#L43

In [2]:
import boto3
import getpass
import pandas as pd
import pyodbc
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns
import psycopg2

from pycaret.clustering import *

In [ ]:
IAM_ROLE = ""

configs = [
    {
        "name": "galaxy"
    },
]

date = datetime.date.today

## 1. Querying the S3 Bucket to get a Dataframe of the Scores
Pros
* a

Cons
* b

In [ ]:

for config in configs:
    url = f"s3://explore-us-curated-data-sci-product-propensity-us-east-1-u8gldf/product-propensity-scores/date={date}/{config['name']}.parquet"

    df = pd.read_parquet(url)


## 2. Redshift Copy
Pros
* a

Cons
* b

In [ ]:
dbname = ""
host = ""
user = ""
password = ""

conn = psycopg2.connect(dbname=dbname, host=host, port='5439', user=user, password=password)
cur = conn.cursor()

# Begin your transaction
cur.execute("begin;")

for config in configs:
    s3_path = f"s3://explore-us-curated-data-sci-product-propensity-us-east-1-u8gldf/product-propensity-scores/date={date}/{config['name']}.parquet"
    cur.execute(f"copy score_data from '{s3_path}' IAM_ROLE '{IAM_ROLE}' parquet;")


# Commit your transaction
cur.execute("commit;")
print("Copy executed fine!")

## 3. Athena Interface
Pros
* a

Cons
* b


In [ ]:
client = boto3.client("athena")

# This will read the data
query_start = client.start_query_execution(
    QueryString = "Select * FROM product_propensity_scores",
    QueryExecutionContext = {
        "Database": "data-sci"
    }
)

# To get the status of the query:
query_execution = client.get_query_execution(QueryExecutionId=query_start['QueryExecutionId'])

# Waits until the query succeeds. Would need a FAILED and CANCELLED check.
while query_execution.Status.State != "SUCCEEDED":
    time.sleep(15)

# To get the data in Lambda. Specify the MaxResults for number of rows per page.
results = client.get_query_results(QueryExecutionId=query_start['QueryExecutionId'], MaxResults=1000000)

next_token = results["NextToken"]
while next_token:
    results = client.get_query_results(QueryExecutionId=query_start['QueryExecutionId'], MaxResults=1000000, NextToken=next_token)
    next_token = results["NextToken"]